In [1]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/requierements.ipynb

# Running functions_bar_igw.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_borrador/IGW_BAR/functions_bar_igw.ipynb

In [ ]:
path_ssh_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar_LR/ssh_bar_"
path_ssh_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw_LR/ssh_igw_"

path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"

In [ ]:
ssh_bar = xr.open_mfdataset(path_ssh_bar + "02.nc")

In [ ]:
ssh_igw = xr.open_mfdataset(path_ssh_igw + "02.nc")

In [ ]:
ssh_igw = xr.open_mfdataset(path_ssh_bar + "02.nc")

In [ ]:
import xarray as xr
import numpy as np
from scipy.interpolate import griddata
import pandas as pd

# Load the ssh_dedac dataset
ssh_dedac = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60_dedac/ssh_dedac_masked_01.nc")
ssh_dedac = ssh_dedac.sel(time_counter=pd.to_datetime('2014-01-31T11:30:00')).load()

# Function to interpolate dataset
def interpolate_to_target_grid(source_dataset, source_var, target_lat, target_lon):
    # Flatten the coordinates and data of the source variable for interpolation
    ENSLAT2D = source_dataset.lat.values
    ENSLON2D = source_dataset.lon.values
    source_data = source_dataset[source_var].isel(time_counter=0).values  # Use the first time step for simplicity

    # Ensure there are no NaNs in the points for interpolation
    valid_points = ~np.isnan(ENSLAT2D) & ~np.isnan(ENSLON2D) & ~np.isnan(source_data)
    points = np.array([ENSLAT2D[valid_points], ENSLON2D[valid_points]]).T
    values = source_data[valid_points]

    # Interpolating using griddata with the linear method
    interpolated_linear = griddata(
        points,
        values,
        (target_lat, target_lon),
        method='linear'
    )

    # Interpolating using griddata with the nearest method to fill NaNs
    interpolated_nearest = griddata(
        points,
        values,
        (target_lat, target_lon),
        method='nearest'
    )

    # Combine the two results, using nearest interpolation where linear interpolation produced NaNs
    combined_interpolated = np.where(np.isnan(interpolated_linear), interpolated_nearest, interpolated_linear)
    
    return combined_interpolated

# Target grid for interpolation
lat2d = ssh_dedac.nav_lat.values
lon2d = ssh_dedac.nav_lon.values

# Interpolating ssh_bar
ssh_bar_interpolated_values = interpolate_to_target_grid(ssh_bar, 'ssh_bar', lat2d, lon2d)

# Creating the ssh_bar DataArray
ssh_bar_interpolated = xr.DataArray(
    data=ssh_bar_interpolated_values,
    dims=ssh_dedac.ssh_dedac.dims,
    coords=ssh_dedac.ssh_dedac.coords
).drop_vars(['latitude', 'longitude'], errors='ignore')

# Interpolating ssh_igw
ssh_igw_interpolated_values = interpolate_to_target_grid(ssh_igw, 'ssh_igw', lat2d, lon2d)

# Creating the ssh_igw DataArray
ssh_igw_interpolated = xr.DataArray(
    data=ssh_igw_interpolated_values,
    dims=ssh_dedac.ssh_dedac.dims,
    coords=ssh_dedac.ssh_dedac.coords
).drop_vars(['latitude', 'longitude'], errors='ignore')

# Create new Datasets with the interpolated DataArrays
ssh_bar_interpolated_dataset = xr.Dataset({'ssh_bar': ssh_bar_interpolated})
ssh_igw_interpolated_dataset = xr.Dataset({'ssh_igw': ssh_igw_interpolated})




In [ ]:
ssh_bar_interpolated_dataset

In [ ]:
plt.figure()
plt.pcolormesh(ssh_bar_interpolated_dataset.ssh_bar)
plt.show()

In [ ]:
path_ssh_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar_LR/ssh_bar_"
path_ssh_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw_LR/ssh_igw_"

path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"

ssh_bar = xr.open_mfdataset(path_ssh_bar + "02.nc").load()
ssh_igw = xr.open_mfdataset(path_ssh_igw + "02.nc").load()


# Load the ssh_dedac dataset
ssh_dedac = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60_dedac/ssh_dedac_masked_01.nc")
ssh_dedac = ssh_dedac.sel(time_counter=pd.to_datetime('2014-01-31T11:30:00')).load()

# Function to interpolate dataset
def interpolate_to_target_grid(source_dataset, source_var, target_lat, target_lon):
    interpolated_data = []

    for t in range(source_dataset.time_counter.size):
        # Flatten the coordinates and data of the source variable for interpolation
        ENSLAT2D = source_dataset.lat.values
        ENSLON2D = source_dataset.lon.values
        source_data = source_dataset[source_var].isel(time_counter=t).values

        # Ensure there are no NaNs in the points for interpolation
        valid_points = ~np.isnan(ENSLAT2D) & ~np.isnan(ENSLON2D) & ~np.isnan(source_data)
        points = np.array([ENSLAT2D[valid_points], ENSLON2D[valid_points]]).T
        values = source_data[valid_points]

        # Interpolating using griddata with the linear method
        interpolated_linear = griddata(
            points,
            values,
            (target_lat, target_lon),
            method='linear'
        )

        # Interpolating using griddata with the nearest method to fill NaNs
        interpolated_nearest = griddata(
            points,
            values,
            (target_lat, target_lon),
            method='nearest'
        )

        # Combine the two results, using nearest interpolation where linear interpolation produced NaNs
        combined_interpolated = np.where(np.isnan(interpolated_linear), interpolated_nearest, interpolated_linear)

        interpolated_data.append(combined_interpolated)

    interpolated_data = np.array(interpolated_data)
    return interpolated_data

# Target grid for interpolation
lat2d = ssh_dedac.nav_lat.values
lon2d = ssh_dedac.nav_lon.values

# Interpolating ssh_bar
ssh_bar_interpolated_values = interpolate_to_target_grid(ssh_bar, 'ssh_bar', lat2d, lon2d)

# Creating the ssh_bar DataArray
ssh_bar_interpolated = xr.DataArray(
    data=ssh_bar_interpolated_values,
    dims=['time_counter', 'y', 'x'],
    coords={'time_counter': ssh_bar.time_counter, 'y': ssh_dedac.y, 'x': ssh_dedac.x, 'nav_lat': (['y', 'x'], lat2d), 'nav_lon': (['y', 'x'], lon2d)}
).drop_vars(['latitude', 'longitude'], errors='ignore')

# Interpolating ssh_igw
ssh_igw_interpolated_values = interpolate_to_target_grid(ssh_igw, 'ssh_igw', lat2d, lon2d)

# Creating the ssh_igw DataArray
ssh_igw_interpolated = xr.DataArray(
    data=ssh_igw_interpolated_values,
    dims=['time_counter', 'y', 'x'],
    coords={'time_counter': ssh_igw.time_counter, 'y': ssh_dedac.y, 'x': ssh_dedac.x, 'nav_lat': (['y', 'x'], lat2d), 'nav_lon': (['y', 'x'], lon2d)}
).drop_vars(['latitude', 'longitude'], errors='ignore')

# Create new Datasets with the interpolated DataArrays
ssh_bar_interpolated_dataset = xr.Dataset({'ssh_bar': ssh_bar_interpolated})
ssh_igw_interpolated_dataset = xr.Dataset({'ssh_igw': ssh_igw_interpolated})

# Save the interpolated datasets if needed
ssh_bar_interpolated_dataset.to_netcdf("/path/to/save/ssh_bar_interpolated.nc")
ssh_igw_interpolated_dataset.to_netcdf("/path/to/save/ssh_igw_interpolated.nc")


Loop for all time steps on a month filee

In [ ]:
path_ssh_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar_LR/ssh_bar_"
path_ssh_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw_LR/ssh_igw_"

path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"

ssh_bar = xr.open_mfdataset(path_ssh_bar + "02.nc").load()
ssh_igw = xr.open_mfdataset(path_ssh_igw + "02.nc").load()

# Load the ssh_dedac dataset
ssh_dedac = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60_dedac/ssh_dedac_masked_01.nc")
ssh_dedac = ssh_dedac.sel(time_counter=pd.to_datetime('2014-01-31T11:30:00')).load()

# Function to interpolate a single time step
def interpolate_single_time_step(t, source_dataset, source_var, target_lat, target_lon):
    # Flatten the coordinates and data of the source variable for interpolation
    ENSLAT2D = source_dataset.lat.values
    ENSLON2D = source_dataset.lon.values
    source_data = source_dataset[source_var].isel(time_counter=t).values

    # Ensure there are no NaNs in the points for interpolation
    valid_points = ~np.isnan(ENSLAT2D) & ~np.isnan(ENSLON2D) & ~np.isnan(source_data)
    points = np.array([ENSLAT2D[valid_points], ENSLON2D[valid_points]]).T
    values = source_data[valid_points]

    # Interpolating using griddata with the linear method
    interpolated_linear = griddata(
        points,
        values,
        (target_lat, target_lon),
        method='linear'
    )

    # Interpolating using griddata with the nearest method to fill NaNs
    interpolated_nearest = griddata(
        points,
        values,
        (target_lat, target_lon),
        method='nearest'
    )

    # Combine the two results, using nearest interpolation where linear interpolation produced NaNs
    combined_interpolated = np.where(np.isnan(interpolated_linear), interpolated_nearest, interpolated_linear)
    
    return combined_interpolated

# Function to interpolate dataset
def interpolate_to_target_grid(source_dataset, source_var, target_lat, target_lon):
    num_cores = -1  # Use all available cores
    interpolated_data = Parallel(n_jobs=num_cores)(delayed(interpolate_single_time_step)(t, source_dataset, source_var, target_lat, target_lon) for t in range(source_dataset.time_counter.size))
    return np.array(interpolated_data)

# Target grid for interpolation
lat2d = ssh_dedac.nav_lat.values
lon2d = ssh_dedac.nav_lon.values

# Interpolating ssh_bar
ssh_bar = xr.open_mfdataset(path_ssh_bar + "02.nc")
ssh_bar_interpolated_values = interpolate_to_target_grid(ssh_bar, 'ssh_bar', lat2d, lon2d)

# Creating the ssh_bar DataArray
ssh_bar_interpolated = xr.DataArray(
    data=ssh_bar_interpolated_values,
    dims=['time_counter', 'y', 'x'],
    coords={'time_counter': ssh_bar.time_counter, 'nav_lat': (['y', 'x'], lat2d), 'nav_lon': (['y', 'x'], lon2d)}
).drop_vars(['y', 'x'], errors='ignore')

# Interpolating ssh_igw
ssh_igw = xr.open_mfdataset(path_ssh_igw + "02.nc")
ssh_igw_interpolated_values = interpolate_to_target_grid(ssh_igw, 'ssh_igw', lat2d, lon2d)

# Creating the ssh_igw DataArray
ssh_igw_interpolated = xr.DataArray(
    data=ssh_igw_interpolated_values,
    dims=['time_counter', 'y', 'x'],
    coords={'time_counter': ssh_igw.time_counter, 'nav_lat': (['y', 'x'], lat2d), 'nav_lon': (['y', 'x'], lon2d)}
).drop_vars(['y', 'x'], errors='ignore')

# Create new Datasets with the interpolated DataArrays
ssh_bar_interpolated_dataset = xr.Dataset({'ssh_bar': ssh_bar_interpolated})
ssh_igw_interpolated_dataset = xr.Dataset({'ssh_igw': ssh_igw_interpolated})

# Save the interpolated datasets
ssh_bar_interpolated_dataset.to_netcdf(path_to_save_bar + "ssh_bar_02.nc")
ssh_igw_interpolated_dataset.to_netcdf(path_to_save_igw + "ssh_igw_02.nc")


Loop for all timessteps on the year

In [2]:
# Base paths
path_ssh_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar_LR/ssh_bar_"
path_ssh_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw_LR/ssh_igw_"

path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"

# Load the ssh_dedac dataset
ssh_dedac = xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60_dedac/ssh_dedac_masked_01.nc")
ssh_dedac = ssh_dedac.sel(time_counter=pd.to_datetime('2014-01-31T11:30:00')).load()

# Function to interpolate a single time step
def interpolate_single_time_step(t, source_dataset, source_var, target_lat, target_lon):
    # Flatten the coordinates and data of the source variable for interpolation
    ENSLAT2D = source_dataset.lat.values
    ENSLON2D = source_dataset.lon.values
    source_data = source_dataset[source_var].isel(time_counter=t).values

    # Ensure there are no NaNs in the points for interpolation
    valid_points = ~np.isnan(ENSLAT2D) & ~np.isnan(ENSLON2D) & ~np.isnan(source_data)
    points = np.array([ENSLAT2D[valid_points], ENSLON2D[valid_points]]).T
    values = source_data[valid_points]

    # Interpolating using griddata with the linear method
    interpolated_linear = griddata(
        points,
        values,
        (target_lat, target_lon),
        method='linear'
    )

    # Interpolating using griddata with the nearest method to fill NaNs
    interpolated_nearest = griddata(
        points,
        values,
        (target_lat, target_lon),
        method='nearest'
    )

    # Combine the two results, using nearest interpolation where linear interpolation produced NaNs
    combined_interpolated = np.where(np.isnan(interpolated_linear), interpolated_nearest, interpolated_linear)
    
    return combined_interpolated

# Function to interpolate dataset
def interpolate_to_target_grid(source_dataset, source_var, target_lat, target_lon):
    num_cores = -1  # Use all available cores
    interpolated_data = Parallel(n_jobs=num_cores)(delayed(interpolate_single_time_step)(t, source_dataset, source_var, target_lat, target_lon) for t in range(source_dataset.time_counter.size))
    return np.array(interpolated_data)

# Target grid for interpolation
lat2d = ssh_dedac.nav_lat.values
lon2d = ssh_dedac.nav_lon.values

# Process each file
for i in range(1, 13):
    # Construct file paths
    file_suffix = f"{i:02d}.nc"
    ssh_bar_path = path_ssh_bar + file_suffix
    ssh_igw_path = path_ssh_igw + file_suffix
    save_ssh_bar_path = os.path.join(path_to_save_bar, f"ssh_bar_{file_suffix}")
    save_ssh_igw_path = os.path.join(path_to_save_igw, f"ssh_igw_{file_suffix}")

    # Load the datasets
    ssh_bar = xr.open_mfdataset(ssh_bar_path).load()
    ssh_igw = xr.open_mfdataset(ssh_igw_path).load()

    # Interpolating ssh_bar
    ssh_bar_interpolated_values = interpolate_to_target_grid(ssh_bar, 'ssh_bar', lat2d, lon2d)

    # Creating the ssh_bar DataArray
    ssh_bar_interpolated = xr.DataArray(
        data=ssh_bar_interpolated_values,
        dims=['time_counter', 'y', 'x'],
        coords={'time_counter': ssh_bar.time_counter, 'nav_lat': (['y', 'x'], lat2d), 'nav_lon': (['y', 'x'], lon2d)}
    ).drop_vars(['y', 'x'], errors='ignore')

    # Interpolating ssh_igw
    ssh_igw_interpolated_values = interpolate_to_target_grid(ssh_igw, 'ssh_igw', lat2d, lon2d)

    # Creating the ssh_igw DataArray
    ssh_igw_interpolated = xr.DataArray(
        data=ssh_igw_interpolated_values,
        dims=['time_counter', 'y', 'x'],
        coords={'time_counter': ssh_igw.time_counter, 'nav_lat': (['y', 'x'], lat2d), 'nav_lon': (['y', 'x'], lon2d)}
    ).drop_vars(['y', 'x'], errors='ignore')

    # Create new Datasets with the interpolated DataArrays
    ssh_bar_interpolated_dataset = xr.Dataset({'ssh_bar': ssh_bar_interpolated})
    ssh_igw_interpolated_dataset = xr.Dataset({'ssh_igw': ssh_igw_interpolated})

    # Save the interpolated datasets
    ssh_bar_interpolated_dataset.to_netcdf(save_ssh_bar_path)
    ssh_igw_interpolated_dataset.to_netcdf(save_ssh_igw_path)


In [ ]:
plt.figure()
plt.pcolormesh(ssh_bar_interpolated_dataset.nav_lat, ssh_bar_interpolated_dataset.nav_lon, ssh_bar_interpolated_dataset.ssh_bar.isel(time_counter=0))
plt.show()